In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import pymysql

In [2]:
# 사이트를 열기
driver = webdriver.Chrome('./chromedriver')
driver.get('http://www.statiz.co.kr/schedule.php?opt=3&sy=2018')

In [5]:
 # 경기 URL 추출
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
url= soup.select('body > div > div.content-wrapper > div > section.content > div > div > div > div > div > div.box-body.no-padding > table > tbody > tr > td > div.hidden-xs > a')

In [6]:
url

[<a href="boxscore.php?date=2018-03-24&amp;stadium=%EA%B3%A0%EC%B2%99%EB%8F%94&amp;hour=14"><span class="badge" style="padding-top:3px;padding-bottom:3px;padding-left:1px;padding-right:1px;color:#ED4C00;background:#EEEEEE;font-size:11px;width:35px;">한화</span><span class="badge" style="margin-top:0px;;width:16px;padding:0px;background:white;font-size:11px;color:black;">3</span><span class="badge" style="margin-top:0px;width:16px;padding:0px;background:white;font-size:11px;color:black;">6</span><span class="badge" style="padding-top:3px;padding-bottom:3px;padding-left:1px;padding-right:1px;width:35px;background:#740000;font-size:11px;color:white;">넥센</span></a>,
 <a href="boxscore.php?date=2018-03-24&amp;stadium=%EB%A7%88%EC%82%B0&amp;hour=14"><span class="badge" style="padding-top:3px;padding-bottom:3px;padding-left:1px;padding-right:1px;color:black;background:#EEEEEE;font-size:11px;width:35px;">LG</span><span class="badge" style="margin-top:0px;;width:16px;padding:0px;background:white;

In [7]:
hrefList=[]
titleList=[]
for i in url:
    href=i.attrs['href'] # href 안에 내용을 다 들고온다.
    title=i.text.replace('\n','').strip() # 문자만 들고온다.
    hrefList.append(href)
    titleList.append(title)

In [8]:
hrefList

['boxscore.php?date=2018-03-24&stadium=%EA%B3%A0%EC%B2%99%EB%8F%94&hour=14',
 'boxscore.php?date=2018-03-24&stadium=%EB%A7%88%EC%82%B0&hour=14',
 'boxscore.php?date=2018-03-24&stadium=%EC%9D%B8%EC%B2%9C%EB%AC%B8%ED%95%99&hour=14',
 'boxscore.php?date=2018-03-24&stadium=%EC%9E%A0%EC%8B%A4&hour=14',
 'boxscore.php?date=2018-03-24&stadium=%EC%B1%94%ED%94%BC%EC%96%B8%EC%8A%A4%ED%95%84%EB%93%9C&hour=14',
 'boxscore.php?date=2018-03-25&stadium=%EA%B3%A0%EC%B2%99%EB%8F%94&hour=14',
 'boxscore.php?date=2018-03-25&stadium=%EB%A7%88%EC%82%B0&hour=14',
 'boxscore.php?date=2018-03-25&stadium=%EC%9D%B8%EC%B2%9C%EB%AC%B8%ED%95%99&hour=14',
 'boxscore.php?date=2018-03-25&stadium=%EC%9E%A0%EC%8B%A4&hour=14',
 'boxscore.php?date=2018-03-25&stadium=%EC%B1%94%ED%94%BC%EC%96%B8%EC%8A%A4%ED%95%84%EB%93%9C&hour=14',
 'boxscore.php?date=2018-03-27&stadium=%EA%B3%A0%EC%B2%99%EB%8F%94&hour=18',
 'boxscore.php?date=2018-03-27&stadium=%EB%A7%88%EC%82%B0&hour=18',
 'boxscore.php?date=2018-03-27&stadium=%EC%9D%B8%

In [9]:
pit=[]
hit=[]
result=[]
for i in hrefList:
    driver.get('http://www.statiz.co.kr/'+i)
    xpath = '/html/body/div/div[1]/div/section[2]/div/div[1]/div/div[3]/div/div[2]/table/tbody/tr/td/a[5]/i'
    driver.find_element_by_xpath(xpath).click()
    time.sleep(3)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    a=soup.select('body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div:nth-child(2) > div > div > div.box-body.no-padding.table-responsive > table > tbody > tr> td:nth-child(2)')
    b=soup.select('body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div:nth-child(2) > div > div > div.box-body.no-padding.table-responsive > table > tbody > tr> td:nth-child(3)')
    c=soup.select('body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div:nth-child(2) > div > div > div.box-body.no-padding.table-responsive > table > tbody > tr> td:nth-child(5)')

    for i in range(len(a)):
        pit.append(a[i].get_text())
        hit.append(b[i].get_text())    
        result.append(c[i].get_text())  
        

df=pd.DataFrame()
df['pit']=pit
df['hit']=hit
df['result']=result
    

In [10]:
df

,pit,hit,result
0,로저스,1 이용규,이용규 : 삼진 아웃
1,로저스,2 양성우,중견수 뜬공
2,로저스,3 송광민,좌익수 안타
3,로저스,4 김태균,우익수 안타
4,로저스,5 하주석,좌익수 안타
5,로저스,6 정근우,유격수 땅볼
6,샘슨,1 서건창,2루수 땅볼
7,샘슨,2 초이스,초이스 : 삼진 아웃
8,샘슨,3 김태완,김태완 : 삼진 아웃
9,로저스,7 호잉,3루수 번트안타


In [3]:
hrefList=[]
titleList=[]
for i in range(3,4):
    driver.get('http://www.statiz.co.kr/schedule.php?opt='+str(i)+'&sy=2019')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    url= soup.select('body > div > div.content-wrapper > div > section.content > div > div > div > div > div > div.box-body.no-padding > table > tbody > tr > td > div.hidden-xs > a')
    for i in url:
        href=i.attrs['href'] # href 안에 내용을 다 들고온다.
        title=i.text.replace('\n','').strip() # 문자만 들고온다.
        hrefList.append(href)
        titleList.append(title)

In [32]:
hrefList

['boxscore.php?date=2019-03-23&stadium=%EB%B6%80%EC%82%B0%EC%82%AC%EC%A7%81&hour=14',
 'boxscore.php?date=2019-03-23&stadium=%EC%9D%B8%EC%B2%9C%EB%AC%B8%ED%95%99&hour=14',
 'boxscore.php?date=2019-03-23&stadium=%EC%9E%A0%EC%8B%A4&hour=14',
 'boxscore.php?date=2019-03-23&stadium=%EC%B0%BD%EC%9B%90NC%ED%8C%8C%ED%81%AC&hour=14',
 'boxscore.php?date=2019-03-23&stadium=%EC%B1%94%ED%94%BC%EC%96%B8%EC%8A%A4%ED%95%84%EB%93%9C&hour=14',
 'boxscore.php?date=2019-03-24&stadium=%EB%B6%80%EC%82%B0%EC%82%AC%EC%A7%81&hour=14',
 'boxscore.php?date=2019-03-24&stadium=%EC%9D%B8%EC%B2%9C%EB%AC%B8%ED%95%99&hour=14',
 'boxscore.php?date=2019-03-24&stadium=%EC%9E%A0%EC%8B%A4&hour=14',
 'boxscore.php?date=2019-03-24&stadium=%EC%B0%BD%EC%9B%90NC%ED%8C%8C%ED%81%AC&hour=14',
 'boxscore.php?date=2019-03-24&stadium=%EC%B1%94%ED%94%BC%EC%96%B8%EC%8A%A4%ED%95%84%EB%93%9C&hour=14',
 'boxscore.php?date=2019-03-26&stadium=%EB%B6%80%EC%82%B0%EC%82%AC%EC%A7%81&hour=18',
 'boxscore.php?date=2019-03-26&stadium=%EC%9D%B8%E

In [33]:
pit=[]
hit=[]
result=[]
for i in hrefList:
    driver.get('http://www.statiz.co.kr/'+i)
    xpath = '/html/body/div/div[1]/div/section[2]/div/div[1]/div/div[3]/div/div[2]/table/tbody/tr/td/a[5]/i'
    driver.find_element_by_xpath(xpath).click()
    time.sleep(3)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    a=soup.select('body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div:nth-child(2) > div > div > div.box-body.no-padding.table-responsive > table > tbody > tr> td:nth-child(2)')
    b=soup.select('body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div:nth-child(2) > div > div > div.box-body.no-padding.table-responsive > table > tbody > tr> td:nth-child(3)')
    c=soup.select('body > div > div.content-wrapper > div > section.content > div > div:nth-child(2) > div:nth-child(2) > div > div > div.box-body.no-padding.table-responsive > table > tbody > tr> td:nth-child(5)')

    for i in range(len(a)):
        pit.append(a[i].get_text())
        hit.append(b[i].get_text())    
        result.append(c[i].get_text())  
        

df=pd.DataFrame()
df['pit']=pit
df['hit']=hit
df['result']=result
    

In [34]:
df

,pit,hit,result
0,레일리,1 서건창,스트라이크낫아웃 출루
1,레일리,2 김하성,2루수 뜬공
2,레일리,3 박병호,2루수 뜬공
3,레일리,4 샌즈,볼넷
4,레일리,5 장영석,2루수 뜬공
5,브리검,1 민병헌,3루수 땅볼
6,브리검,2 손아섭,우익수 뜬공
7,브리검,3 전준우,3루수 땅볼
8,레일리,6 임병욱,투수 땅볼 실책[실책포함]
9,레일리,7 허정협,유격수 뜬공


In [35]:
df.to_csv('./detail_2019.csv',index=False,encoding='utf-8')